## Importing Libraries

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
#from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.master("local[*]").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [4]:
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

## Loading data

In [ ]:
data = spark.read.csv('train.csv', header = True, inferSchema = True)
data.show(5)

## Preparing the data

> ### Splitting the data

We will use 80% of data as training data, and 20% as testing data.

In [ ]:
splits = data.randomSplit([0.8,0.2])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

> ### Defining our pipeline